In [1]:
# Import Dependencies
import numpy as np

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import pandas as pd

In [2]:
# Import Csv

file_to_load = 'Resources/Mental_Health_Centers.csv'

# Convert to DF
centers_df = pd.read_csv(file_to_load)
centers_df

,X,Y,OBJECTID,source,ext_id,cat1,cat2,cat3,org_name,Name,...,zip,link,use_type,latitude,longitude,date_updated,email,dis_status,POINT_X,POINT_Y
0,-118.242583,33.924375,61792,NaN,NaN,Health and Mental Health,Mental Health Centers,NGA,Augustues F. Hawkins Mental Health Center,Augustus F. Hawkins Mental Health Center,...,90059,http://egis3.lacounty.gov/lms/?p=68253,publish,33.924375,-118.242582,2013/08/21 11:15:59+00,NaN,NaN,6.488077e+06,1.794947e+06
1,-118.143896,33.892282,61794,NaN,NaN,Health and Mental Health,Mental Health Centers,NaN,La Paz Gero-Psychiatric Center,La Paz Gero-Psychiatric Center,...,90723,http://egis3.lacounty.gov/lms/?p=68255,publish,33.892282,-118.143896,2013/08/21 11:18:46+00,NaN,Active,6.517999e+06,1.783211e+06
2,-118.143896,33.892282,61795,NaN,NaN,Health and Mental Health,Mental Health Centers,NGA,La Paz Gero-Psychiatric Center,La Paz Gero-Psychiatric Center,...,90723,http://egis3.lacounty.gov/lms/?p=68255,publish,33.892282,-118.143896,2013/08/21 11:18:46+00,NaN,Active,6.517999e+06,1.783211e+06
3,-118.266952,34.007385,61797,NaN,NaN,Health and Mental Health,Mental Health Centers,NaN,Kedren Mental Health Center,Kedren Mental Health Center,...,90011,http://egis3.lacounty.gov/lms/?p=68259,publish,34.007385,-118.266952,2013/09/10 15:09:10+00,NaN,NaN,6.480764e+06,1.825175e+06
4,-118.266952,34.007385,61799,NaN,NaN,Health and Mental Health,Mental Health Centers,NGA,Kedren Mental Health Center,Kedren Mental Health Center,...,90011,http://egis3.lacounty.gov/lms/?p=68259,publish,34.007385,-118.266952,2013/09/10 15:09:10+00,NaN,NaN,6.480764e+06,1.825175e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,-118.460378,34.205132,72517,LA County Department of Mental Health,19DMA,Health and Mental Health,Mental Health Centers,NaN,LOS ANGELES COUNTY DEPARTMENT OF MENTAL HEALTH,HSCFS VALERIO ES,...,91405,http://egis3.lacounty.gov/health_lms/?p=2009368,publish,34.205132,-118.460378,2020/05/08 14:28:39+00,NaN,NaN,6.422472e+06,1.897351e+06
912,-118.006301,34.115791,72518,LA County Department of Mental Health,19DPA,Health and Mental Health,Mental Health Centers,NaN,LOS ANGELES COUNTY DEPARTMENT OF MENTAL HEALTH,DFYS PLYMOUTH ELEM SCH,...,91016,http://egis3.lacounty.gov/health_lms/?p=2009370,publish,34.115791,-118.006301,2020/05/08 15:11:14+00,NaN,NaN,6.559763e+06,1.864519e+06
913,-117.952158,34.144620,72519,LA County Department of Mental Health,19DQA,Health and Mental Health,Mental Health Centers,NaN,LOS ANGELES COUNTY DEPARTMENT OF MENTAL HEALTH,DFYS ROYAL OAKS STEM ACAD,...,91008,http://egis3.lacounty.gov/health_lms/?p=2009373,publish,34.144620,-117.952158,2020/05/08 17:05:19+00,NaN,NaN,6.576146e+06,1.875014e+06
914,-118.128818,34.093074,72520,LA County Department of Mental Health,19D6A,Health and Mental Health,Mental Health Centers,NaN,LOS ANGELES COUNTY DEPARTMENT OF MENTAL HEALTH,HSCFS ALHAMBRA HS,...,91801,http://egis3.lacounty.gov/health_lms/?p=2009379,publish,34.093074,-118.128818,2020/05/13 15:52:08+00,NaN,NaN,6.522669e+06,1.856277e+06


In [3]:
# Examining the columns
centers_df.columns

Index(['X', 'Y', 'OBJECTID', 'source', 'ext_id', 'cat1', 'cat2', 'cat3',
       'org_name', 'Name', 'addrln1', 'addrln2', 'city', 'state', 'hours',
       'phones', 'url', 'info1', 'info2', 'post_id', 'description', 'zip',
       'link', 'use_type', 'latitude', 'longitude', 'date_updated', 'email',
       'dis_status', 'POINT_X', 'POINT_Y'],
      dtype='object')

In [4]:
# Drop Null columns
centers_df = centers_df.drop(['email', 'source', 'link', 'use_type', 'cat3', 'description', 'ext_id', 'addrln1', 'addrln2', 'info1', 'info2', 'hours', 'phones'],axis = 1)
centers_df.head(10)

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,url,post_id,zip,latitude,longitude,date_updated,dis_status,POINT_X,POINT_Y
0,-118.242583,33.924375,61792,Health and Mental Health,Mental Health Centers,Augustues F. Hawkins Mental Health Center,Augustus F. Hawkins Mental Health Center,Los Angeles,CA,NaN,68253,90059,33.924375,-118.242582,2013/08/21 11:15:59+00,NaN,6.488077e+06,1.794947e+06
1,-118.143896,33.892282,61794,Health and Mental Health,Mental Health Centers,La Paz Gero-Psychiatric Center,La Paz Gero-Psychiatric Center,Paramount,CA,NaN,68255,90723,33.892282,-118.143896,2013/08/21 11:18:46+00,Active,6.517999e+06,1.783211e+06
2,-118.143896,33.892282,61795,Health and Mental Health,Mental Health Centers,La Paz Gero-Psychiatric Center,La Paz Gero-Psychiatric Center,Paramount,CA,NaN,68255,90723,33.892282,-118.143896,2013/08/21 11:18:46+00,Active,6.517999e+06,1.783211e+06
3,-118.266952,34.007385,61797,Health and Mental Health,Mental Health Centers,Kedren Mental Health Center,Kedren Mental Health Center,"Los Angeles,",CA,NaN,68259,90011,34.007385,-118.266952,2013/09/10 15:09:10+00,NaN,6.480764e+06,1.825175e+06
4,-118.266952,34.007385,61799,Health and Mental Health,Mental Health Centers,Kedren Mental Health Center,Kedren Mental Health Center,"Los Angeles,",CA,NaN,68259,90011,34.007385,-118.266952,2013/09/10 15:09:10+00,NaN,6.480764e+06,1.825175e+06
5,-118.264830,33.930407,61801,Health and Mental Health,Mental Health Centers,Alafia Mental Health Center,Alafia Mental Health Center,Los Angeles,CA,NaN,68263,90061,33.930407,-118.264830,2013/09/10 15:21:11+00,NaN,6.481333e+06,1.797160e+06
6,-118.264830,33.930407,61803,Health and Mental Health,Mental Health Centers,Alafia Mental Health Center,Alafia Mental Health Center,Los Angeles,CA,NaN,68263,90061,33.930407,-118.264830,2013/09/10 15:21:11+00,NaN,6.481333e+06,1.797160e+06
7,-118.338235,34.007146,61805,Health and Mental Health,Mental Health Centers,Alafia Mental Health Institution,Alafia Mental Health Institution,Los Angeles,CA,NaN,68269,90061,34.007146,-118.338235,2013/09/10 15:37:02+00,NaN,6.459160e+06,1.825153e+06
8,-118.338235,34.007146,61806,Health and Mental Health,Mental Health Centers,Alafia Mental Health Institution,Alafia Mental Health Institution,Los Angeles,CA,NaN,68269,90061,34.007146,-118.338235,2013/09/10 15:37:02+00,NaN,6.459160e+06,1.825153e+06
9,-118.226908,33.923452,61808,Health and Mental Health,Mental Health Centers,Barbour & Floyd Medical Association,Barbour & Floyd Medical Association,Lynwood,CA,NaN,68274,90262,33.923452,-118.226908,2013/08/21 13:40:53+00,NaN,6.492831e+06,1.794601e+06


In [5]:
centers_df.count()

X               916
Y               916
OBJECTID        916
cat1            916
cat2            916
org_name        916
Name            916
city            916
state           913
url               3
post_id         916
zip             913
latitude        916
longitude       916
date_updated    916
dis_status        2
POINT_X         916
POINT_Y         916
dtype: int64

In [6]:
# Dropping additional columns
centers_df = centers_df.drop(['url', 'dis_status'], axis=1)
centers_df.head(10)

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,post_id,zip,latitude,longitude,date_updated,POINT_X,POINT_Y
0,-118.242583,33.924375,61792,Health and Mental Health,Mental Health Centers,Augustues F. Hawkins Mental Health Center,Augustus F. Hawkins Mental Health Center,Los Angeles,CA,68253,90059,33.924375,-118.242582,2013/08/21 11:15:59+00,6.488077e+06,1.794947e+06
1,-118.143896,33.892282,61794,Health and Mental Health,Mental Health Centers,La Paz Gero-Psychiatric Center,La Paz Gero-Psychiatric Center,Paramount,CA,68255,90723,33.892282,-118.143896,2013/08/21 11:18:46+00,6.517999e+06,1.783211e+06
2,-118.143896,33.892282,61795,Health and Mental Health,Mental Health Centers,La Paz Gero-Psychiatric Center,La Paz Gero-Psychiatric Center,Paramount,CA,68255,90723,33.892282,-118.143896,2013/08/21 11:18:46+00,6.517999e+06,1.783211e+06
3,-118.266952,34.007385,61797,Health and Mental Health,Mental Health Centers,Kedren Mental Health Center,Kedren Mental Health Center,"Los Angeles,",CA,68259,90011,34.007385,-118.266952,2013/09/10 15:09:10+00,6.480764e+06,1.825175e+06
4,-118.266952,34.007385,61799,Health and Mental Health,Mental Health Centers,Kedren Mental Health Center,Kedren Mental Health Center,"Los Angeles,",CA,68259,90011,34.007385,-118.266952,2013/09/10 15:09:10+00,6.480764e+06,1.825175e+06
5,-118.264830,33.930407,61801,Health and Mental Health,Mental Health Centers,Alafia Mental Health Center,Alafia Mental Health Center,Los Angeles,CA,68263,90061,33.930407,-118.264830,2013/09/10 15:21:11+00,6.481333e+06,1.797160e+06
6,-118.264830,33.930407,61803,Health and Mental Health,Mental Health Centers,Alafia Mental Health Center,Alafia Mental Health Center,Los Angeles,CA,68263,90061,33.930407,-118.264830,2013/09/10 15:21:11+00,6.481333e+06,1.797160e+06
7,-118.338235,34.007146,61805,Health and Mental Health,Mental Health Centers,Alafia Mental Health Institution,Alafia Mental Health Institution,Los Angeles,CA,68269,90061,34.007146,-118.338235,2013/09/10 15:37:02+00,6.459160e+06,1.825153e+06
8,-118.338235,34.007146,61806,Health and Mental Health,Mental Health Centers,Alafia Mental Health Institution,Alafia Mental Health Institution,Los Angeles,CA,68269,90061,34.007146,-118.338235,2013/09/10 15:37:02+00,6.459160e+06,1.825153e+06
9,-118.226908,33.923452,61808,Health and Mental Health,Mental Health Centers,Barbour & Floyd Medical Association,Barbour & Floyd Medical Association,Lynwood,CA,68274,90262,33.923452,-118.226908,2013/08/21 13:40:53+00,6.492831e+06,1.794601e+06


In [7]:
# Find and drop remaining null values

centers_df.isnull().sum()

X               0
Y               0
OBJECTID        0
cat1            0
cat2            0
org_name        0
Name            0
city            0
state           3
post_id         0
zip             3
latitude        0
longitude       0
date_updated    0
POINT_X         0
POINT_Y         0
dtype: int64

In [8]:
# Drop rows with null values

centers_df = centers_df.dropna(subset=['state', 'zip'])
centers_df.isnull().sum()

X               0
Y               0
OBJECTID        0
cat1            0
cat2            0
org_name        0
Name            0
city            0
state           0
post_id         0
zip             0
latitude        0
longitude       0
date_updated    0
POINT_X         0
POINT_Y         0
dtype: int64

In [9]:
centers_df.count()
centers_df.head(10)

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,post_id,zip,latitude,longitude,date_updated,POINT_X,POINT_Y
0,-118.242583,33.924375,61792,Health and Mental Health,Mental Health Centers,Augustues F. Hawkins Mental Health Center,Augustus F. Hawkins Mental Health Center,Los Angeles,CA,68253,90059,33.924375,-118.242582,2013/08/21 11:15:59+00,6.488077e+06,1.794947e+06
1,-118.143896,33.892282,61794,Health and Mental Health,Mental Health Centers,La Paz Gero-Psychiatric Center,La Paz Gero-Psychiatric Center,Paramount,CA,68255,90723,33.892282,-118.143896,2013/08/21 11:18:46+00,6.517999e+06,1.783211e+06
2,-118.143896,33.892282,61795,Health and Mental Health,Mental Health Centers,La Paz Gero-Psychiatric Center,La Paz Gero-Psychiatric Center,Paramount,CA,68255,90723,33.892282,-118.143896,2013/08/21 11:18:46+00,6.517999e+06,1.783211e+06
3,-118.266952,34.007385,61797,Health and Mental Health,Mental Health Centers,Kedren Mental Health Center,Kedren Mental Health Center,"Los Angeles,",CA,68259,90011,34.007385,-118.266952,2013/09/10 15:09:10+00,6.480764e+06,1.825175e+06
4,-118.266952,34.007385,61799,Health and Mental Health,Mental Health Centers,Kedren Mental Health Center,Kedren Mental Health Center,"Los Angeles,",CA,68259,90011,34.007385,-118.266952,2013/09/10 15:09:10+00,6.480764e+06,1.825175e+06
5,-118.264830,33.930407,61801,Health and Mental Health,Mental Health Centers,Alafia Mental Health Center,Alafia Mental Health Center,Los Angeles,CA,68263,90061,33.930407,-118.264830,2013/09/10 15:21:11+00,6.481333e+06,1.797160e+06
6,-118.264830,33.930407,61803,Health and Mental Health,Mental Health Centers,Alafia Mental Health Center,Alafia Mental Health Center,Los Angeles,CA,68263,90061,33.930407,-118.264830,2013/09/10 15:21:11+00,6.481333e+06,1.797160e+06
7,-118.338235,34.007146,61805,Health and Mental Health,Mental Health Centers,Alafia Mental Health Institution,Alafia Mental Health Institution,Los Angeles,CA,68269,90061,34.007146,-118.338235,2013/09/10 15:37:02+00,6.459160e+06,1.825153e+06
8,-118.338235,34.007146,61806,Health and Mental Health,Mental Health Centers,Alafia Mental Health Institution,Alafia Mental Health Institution,Los Angeles,CA,68269,90061,34.007146,-118.338235,2013/09/10 15:37:02+00,6.459160e+06,1.825153e+06
9,-118.226908,33.923452,61808,Health and Mental Health,Mental Health Centers,Barbour & Floyd Medical Association,Barbour & Floyd Medical Association,Lynwood,CA,68274,90262,33.923452,-118.226908,2013/08/21 13:40:53+00,6.492831e+06,1.794601e+06


In [10]:
centers_df.count()
# Only include mental health centers listed in Los Angeles
#la_centers_df = centers_df.loc[centers_df['city']== 'Los Angeles']
#la_centers_df.head(10)


X               910
Y               910
OBJECTID        910
cat1            910
cat2            910
org_name        910
Name            910
city            910
state           910
post_id         910
zip             910
latitude        910
longitude       910
date_updated    910
POINT_X         910
POINT_Y         910
dtype: int64

In [12]:
# Sorting the columns by zip

centers_sorted = centers_df.sort_values(by = 'zip', ascending = False)
centers_sorted

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,post_id,zip,latitude,longitude,date_updated,POINT_X,POINT_Y
834,-118.043903,34.571358,72440,Health and Mental Health,Mental Health Centers,"JWCH Institute, Inc. – Wesley Palmdale East","JWCH Institute, Inc. – Wesley Palmdale East",Palmdale,CA,2007348,93552,34.571358,-118.043903,2020/02/19 11:37:40+00,6.548454e+06,2.030312e+06
612,-118.137915,34.582571,72218,Health and Mental Health,Mental Health Centers,LA County Department of Mental Health,TESSIE CLEVELAND COM SERVICES,PALMDALE,CA,2005585,93551,34.582571,-118.137915,2018/03/06 09:13:11+00,6.520159e+06,2.034418e+06
175,-118.147850,34.609340,71781,Health and Mental Health,Mental Health Centers,LA County Department of Mental Health,CHILD AND FAM GUIDANCE CENTER-PALMDALE,PALMDALE,CA,1000153,93551,34.609340,-118.147850,2017/02/08 10:49:24+00,6.517183e+06,2.044164e+06
347,-118.140270,34.583760,71953,Health and Mental Health,Mental Health Centers,LA County Department of Mental Health,OPTIMIST YOUTH HOMES-PALMDALE,PALMDALE,CA,1000335,93551,34.583760,-118.140270,2016/05/30 18:53:23+00,6.519451e+06,2.034852e+06
236,-118.100880,34.579980,71842,Health and Mental Health,Mental Health Centers,LA County Department of Mental Health,PALMDALE DISCOVERY CENTER,PALMDALE,CA,1000221,93550,34.579980,-118.100880,2016/05/30 18:53:23+00,6.531305e+06,2.033462e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,-118.237404,33.943524,72482,Health and Mental Health,Mental Health Centers,LOS ANGELES COUNTY DEPARTMENT OF MENTAL HEALTH,CHILDREN'S INSTITUTE INC,LOS ANGELES,CA,2008687,90002,33.943524,-118.237404,2020/01/22 14:34:07+00,6.489665e+06,1.801913e+06
818,-118.247272,33.943954,72424,Health and Mental Health,Mental Health Centers,LOS ANGELES COUNTY DEPARTMENT OF MENTAL HEALTH,CII - WATTS CAMPUS,LOS ANGELES,CA,2007272,90002,33.943954,-118.247272,2020/05/22 17:00:14+00,6.486672e+06,1.802076e+06
444,-118.247720,33.966420,72050,Health and Mental Health,Mental Health Centers,LA County Department of Mental Health,SSG TESSIE CLEVELAND CSC,LOS ANGELES,CA,1000435,90001,33.966420,-118.247720,2016/05/30 18:53:23+00,6.486556e+06,1.810252e+06
416,-118.247720,33.966420,72022,Health and Mental Health,Mental Health Centers,LA County Department of Mental Health,TESSIE CLEVELAND COMMUNITY SERVICES,LOS ANGELES,CA,1000406,90001,33.966420,-118.247720,2018/05/17 13:34:10+00,6.486556e+06,1.810252e+06


In [14]:
centers_sorted.count()

X               910
Y               910
OBJECTID        910
cat1            910
cat2            910
org_name        910
Name            910
city            910
state           910
post_id         910
zip             910
latitude        910
longitude       910
date_updated    910
POINT_X         910
POINT_Y         910
dtype: int64

In [16]:
# Export Sorted Data to CSV

centers_sorted.to_csv('Cleaned_Data/mh_centers_cleaned.csv')